# Programaciò paral·lela
## Lab 2: Color conversion in CUDA

Following the previous laboratory based on OpenMP, now we are going to work on the same simple algorithm, in CUDA.

The goal of this lab is to learn the basics of CUDA kernels, and CUDA Host code.

This new ipython notebook format, will make things easyer since explanation and code will coexist in the same document, and it will be very clear what you need to do.

Additionally, having the Colab platform available with NVIDIA GPU's makes it simpler than ever. You can learn CUDA from any system, Mac, Windows, Linux, and any hardware, Intel, AMD, NVIDIA, and possibliy even ARM on tablets. You only need a web browser compatible with Colab.

## Structure of the Lab
You already know the algorithm from the previous lab, but you may not be familiar with this environment.

First we will try to understand a bit this environment, and then we will explain section by section what you have to do. There are 6 sections.

You will have to complete code in the 6 sections, and perform experiments and comment the results in a separated report. Use tables and figures that support both the results you collected and the arguments you make to justify the results.

## The collab environment for CUDA

First of all, you should know that we are executing an iPython notebook in a Google Colab session. The notebook is preconfigured with the type of execution environment we need, a GPU execution environment. But the files we generate, and the pluggins we install or enable, reside on the Google Colab session. All this will be removed when we exit the session either manually or implicitly by closing the broser.

In order to have a GPU available when creating a new notebook, you only have to select the execution environment.
In Spanish, go to "Entorno de ejecución->Cambiar tipo de entorno de ejecución" and then select GPU.

But as we already mentioned, this notebook is already configured, so you don't need to do it again.

Now, the first thing we will see is that we have the nvcc compiler. We can call many bash commands with ! as the first character, in a code block. Next you will find a code block with a call to nvcc (the nvidia CUDA compiler) with a flag that asks for the compiler version.

Click on the block and then a play button will appear on the left. Click on the play button. 


In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


You can also execute it by placing the cursor inside the code block and pressing Shift+Enter

Next you need to install a pluggin, that does not come with the notebook. In the following code block you have the code line to be executed. You will have to execute this code every time you open the notebook.

In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-__elu028
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-__elu028
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=ecaae3cbfbb2079039e78526dca8650cf69c6d947408fd6104bbe07d5d6dc68e
  Stored in directory: /tmp/pip-ephem-wheel-cache-4p47z2zl/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


Now, you can compile and execute CUDA code, just by puting the same code you would put ina .cu file, just by adding %%cu as the first line.

Next, you have a code example. Try it! Read the comments to help you understand it. It will be very useful for the tasks you have to do.

In [3]:
%%cu
#include <stdio.h>
#include <stdlib.h>

// Function to print the cuda errors
void cuCheck(cudaError_t err) {
    if(err!=cudaSuccess) {
          printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
    }
}

// This macros help with capturing the possible cuda errors and printing
// the error name to help the developer.
// Kernels are always asynchronous with respect to the Host, so they don't return
// any value. Then to see if any error happened, you should call cudaGetLastError
// and pass the result to cuCheck()
// Use the macros instead, to make it simpler.
#define CU_CHECK(a) cuCheck(a)
#define CU_CHECK_LAST_ERROR cuCheck(cudaGetLastError())

// Device code or Kernel
__global__ void add(int a, int b, int* __restrict d_c) {
    *d_c = a * b;
}

// Host code
int main() {
    
    // Host variables a & b
    int a = 3, b = 5, h_c = 0;

    // Host variable that will store a Device pointer wich we can later on 
    // download to the Host.
    // As this variable will contain pointers that are only valid in
    // the Device (the GPU) it will be invalid to access them from
    // Host code. We only can use them in the right cuda API calls
    // or inside a cuda Kernel.
    // So in this part of the code you won't be able to do d_c[0], for instance
    int *d_c;

    // Size of the data contained in variables a, b and c.
    int dataSize = sizeof(int);

    // Reserve Device memory using the cuda API
    // cudaMalloc will place a Device pointer inside d_c.
    CU_CHECK(
        cudaMalloc((void **)&d_c, dataSize)
    );

    // Launch add() kernel on GPU
    // Notice that a and b are not pointers. Therefore the kernel call will
    // copy their values but the variables inside the kernel will not be the same.
    // If we modify a and b inside the kernel, it will not change a and b in this
    // Host code. This, indeed is the same behavior as any C/C++ function call.
    // In the case of d_c, it will copy the pointer contained in d_c, 
    // so we will be able to modify the contents of d_c from the kernel. But to read 
    // them from this Host code, we will have to do something else.
    add<<<1,1>>>(a, b, d_c);
    CU_CHECK_LAST_ERROR;

    CU_CHECK(
        // Copy result back to host
        cudaMemcpy(&h_c, d_c, dataSize, cudaMemcpyDeviceToHost)
    );

    printf("Result of multiplying %d * %d is %d\n\n",a,b,h_c);

    int numDevs=0;
    CU_CHECK(
        cudaGetDeviceCount(&numDevs)
    );

    cudaDeviceProp prop;
    CU_CHECK(
        cudaGetDeviceProperties(&prop, 0)
    );
    printf("Device Number: %d\n", 0);
    printf("  Device name: %s\n", prop.name);
    printf("  Memory Clock Rate (KHz): %d\n",
          prop.memoryClockRate);
    printf("  Memory Bus Width (bits): %d\n",
          prop.memoryBusWidth);
    printf("  Peak Memory Bandwidth (GB/s): %f\n\n",
          2.0*prop.memoryClockRate*(prop.memoryBusWidth/8)/1.0e6);
    printf("Num devices %d\n", numDevs);
    // Cleanup
    CU_CHECK(
      cudaFree(d_c)
    );
    return 0;
}

Result of multiplying 3 * 5 is 15

Device Number: 0
  Device name: Tesla P100-PCIE-16GB
  Memory Clock Rate (KHz): 715000
  Memory Bus Width (bits): 4096
  Peak Memory Bandwidth (GB/s): 732.160000

Num devices 1



Ok, cool! But what if I want to have some code in a .h file, the cuda kernels in an other .h file, and include both so that I can reuse code?

Ok, let's try to put the macros and cuCheck function in a .h file, the kernel in an other .h file and the rest in a .cu file, and compile and execute everything. 

In [4]:
%%cuda --name utils.h
#include <iostream>

void cuCheck(cudaError_t err, const std::string message = "CUDA error:") {
  if(err!=cudaSuccess) {
    std::cout << message << " ERROR " << cudaGetErrorString(err) << std::endl;
  }
}
#define CU_CHECK(a) cuCheck(a)
#define CU_CHECK2(a, b) cuCheck(a, b)
#define CU_CHECK_LAST_ERROR cuCheck(cudaGetLastError())

'File written in /content/src/utils.h'

In [5]:
%%cuda --name kernels.h
__global__ void add(int a, int b, int* __restrict d_c) {
    *d_c = a * b;
}

'File written in /content/src/kernels.h'

In [6]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include "/content/src/utils.h"
#include "/content/src/kernels.h"

// Host code
int main() {
    int a = 3, b = 5, h_c = 0;
    int *d_c;
    int dataSize = sizeof(int);
    CU_CHECK(cudaMalloc((void **)&d_c, dataSize));
    add<<<1,1>>>(a, b, d_c);
    CU_CHECK_LAST_ERROR;
    CU_CHECK(cudaMemcpy(&h_c, d_c, dataSize, cudaMemcpyDeviceToHost));
    int numDevs=0;
    CU_CHECK(cudaGetDeviceCount(&numDevs));
    cudaDeviceProp prop;
    CU_CHECK(cudaGetDeviceProperties(&prop, 0));
    printf("Device Number: %d\n", 0);
    printf("  Device name: %s\n", prop.name);
    printf("  Memory Clock Rate (KHz): %d\n",
          prop.memoryClockRate);
    printf("  Memory Bus Width (bits): %d\n",
          prop.memoryBusWidth);
    printf("  Peak Memory Bandwidth (GB/s): %f\n\n",
          2.0*prop.memoryClockRate*(prop.memoryBusWidth/8)/1.0e6);
    printf("Num devices %d\n", numDevs);
    printf("Result of multiplying %d * %d is %d\n",a,b,h_c);
    CU_CHECK(cudaFree(d_c));
    return 0;
}

Device Number: 0
  Device name: Tesla P100-PCIE-16GB
  Memory Clock Rate (KHz): 715000
  Memory Bus Width (bits): 4096
  Peak Memory Bandwidth (GB/s): 732.160000

Num devices 1
Result of multiplying 3 * 5 is 15



VERY IMPORTANT!!! On each Colab session, the GPU that Google Colab provides can be different. Take it into account when you perform experiments, so that you compare results for the same GPU.

If you have to repeat all the experiments, well, it's not that hard, just click play in all the code blocks one by one.

Great!! Now we can start the lab :-D

##Section 1:

Try to complete the following code, and make it compile. Remember that you have some slides and documents, and the CUDA API specification in the following link: https://docs.nvidia.com/cuda/cuda-runtime-api/index.html

Also, you can search in Google, things like "How to allocate CUDA memory". And so on. Be brave! Is not so difficult.

### First, complete the allocation functions.

In [7]:
%%cuda --name memory_functions.h
void allocGPUData(int width, int height, uchar3** d_brg, uchar4** d_rgba){
  // Alloc gpu pointers
  CU_CHECK2(cudaMalloc(d_brg, sizeof(uchar3)*width*height), "Alloc d_brg:");
  // Can you finish this one? Replace cudaSucces with the proper cuda API call
  CU_CHECK2(cudaMalloc(d_rgba, sizeof(uchar4)*width*height), "Alloc d_rgba:");
}
void copyAndInitializeGPUData(int width, int height, uchar3* h_brg, uchar3* d_brg, uchar4* d_rgba, cudaStream_t stream=0) {
  // Copy data to GPU
  CU_CHECK2(cudaMemcpy(d_brg, h_brg, width*height*sizeof(uchar3), cudaMemcpyHostToDevice), "Copy h_brg to d_brg:");
  // Init output buffer to 0
  CU_CHECK2(cudaMemset(d_rgba, 0, width*height*sizeof(uchar4)), "Memset d_rgba:");
}
void freeCUDAPointers(uchar3* d_brg, uchar4* d_rgba) {
  // Free cuda pointers. Replace the cudaErrorInvalidValue flag
  // with the proper cuda API call, to free the GPU pointers
  CU_CHECK2(cudaFree(d_brg), "Cuda free d_bgr:");
  CU_CHECK2(cudaFree(d_rgba), "Cuda free d_rgba:");
  // Clean GPU device
  CU_CHECK2(cudaDeviceReset(), "Cuda device reset:");
}

'File written in /content/src/memory_functions.h'

### When completed, test that they work with this small main function. If you execute it without completing the previous code, it will show some errors.

In [8]:
%%cu
#include <cuda.h>
#include "/content/src/utils.h"
#include "/content/src/memory_functions.h"

#define WIDTH 10
#define HEIGHT 10

int main() {

  uchar3 *h_brg, *d_brg;
  uchar4 *h_rgba, *d_rgba;

  h_brg = (uchar3*)malloc(sizeof(uchar3)*WIDTH*HEIGHT);
  h_rgba = (uchar4*)malloc(sizeof(uchar4)*WIDTH*HEIGHT);

  allocGPUData(WIDTH, HEIGHT, &d_brg, &d_rgba);
  copyAndInitializeGPUData(WIDTH, HEIGHT, h_brg, d_brg, d_rgba);
  freeCUDAPointers(d_brg, d_rgba);

  return 0;
}

### Ok, now that we have the allocation, copy and free functions implemented, let's continue with the CPU function that will check the results. This one it's already implemented, you only need to click play to have it available.

In [9]:
%%cuda --name check_results.h
bool checkResults(uchar4* rgba, uchar3* brg, int size) {
  bool correct = true;
  for (int i=0; i < size; ++i) {
    // In case you want to see actual values
    if (i==3) {
      unsigned char x, y, z, w;
      x = rgba[i].x;
      y = rgba[i].y;
      z = rgba[i].z;
      w = rgba[i].w;
      std::cout << "First position x=" << (unsigned int)x << " y=" << (unsigned int)y << " z=" << (unsigned int)z << " w=" << (unsigned int)w << std::endl;
    }
    correct &= rgba[i].x == brg[i].y;
    correct &= rgba[i].y == brg[i].z;
    correct &= rgba[i].z == brg[i].x;
    correct &= rgba[i].w == 255;
  }
  return correct;
}

'File written in /content/src/check_results.h'

### Now the interesting part, the kernel and the code to configure and launch it. The kernel it's almost exactly the same code as the OpenMP lab, only we replaced the forloops with something that you need to implement.

Remember, that we have threads with indexes. This indexes are used to tell each CUDA thread, which data do they have to read or write.

The structs that contain those indexes are in the documentation you have available in campusvirtual. Please check the docs.

In [10]:
%%cuda --name cuda_launcher.h
// BIDIMENSIONAL KERNEL
__global__ void convertBRG2RGBA(uchar3 *brg, uchar4* rgba, int width, int height) {
  int x = threadIdx.x + (blockIdx.x * blockDim.x); //Use the thread id and block id's to compute x 
  int y = threadIdx.y + (blockIdx.y * blockDim.y); //Use the thread id and block id's to compute y

	// Protection to avoid segmentation fault
	if (x < width && y < height) {	
	    rgba[width * y + x].x = brg[width * y + x].y;
	    rgba[width * y + x].y = brg[width * y + x].z;
	    rgba[width * y + x].z = brg[width * y + x].x;
	    rgba[width * y + x].w = 255;
	}
}

void executeKernelconvertBRG2RGBA(int width, int height, uchar3* d_brg, uchar4* d_rgba, int numIters, cudaStream_t stream=0) {
  // Execute the GPU kernel
  dim3 block(256, 4, 1);
  dim3 grid(ceil(width/(float)block.x),ceil(height/(float)block.y) , 1);

  // A trick to avoid some undesired optimizations, that will not happen in real applications
  uchar3* d_dataVector[2];
  uchar3* d_secondData;
  CU_CHECK2(cudaMalloc(&d_secondData, width * height * sizeof(uchar3)), "cudaMalloc in executeKernelconvertBRG2RGBA");

  d_dataVector[0] = d_brg;
  d_dataVector[1] = d_secondData;

  CU_CHECK2(cudaMemcpyAsync(d_secondData, d_brg, width * height * sizeof(uchar3), cudaMemcpyDeviceToDevice, stream), "cudaMemcpyAsync in executeKernelconvertBRG2RGBA");

  auto t1 = std::chrono::high_resolution_clock::now();
  for (int i=0; i<numIters; ++i) {
    convertBRG2RGBA<<<grid, block, 0, stream>>>(d_dataVector[i%2], d_rgba, width, height);
  }
  CU_CHECK2(cudaDeviceSynchronize(), "cudaDeviceSynchronize:");
  auto t2 = std::chrono::high_resolution_clock::now();
  auto duration = std::chrono::duration_cast<std::chrono::microseconds>( t2 - t1 ).count();
  std::cout << "convertBRG2RGBA time for " << numIters << " iterations = "<< duration << "us" << std::endl;

  CU_CHECK2(cudaFree(d_secondData), "cudaFree in executeKernelconvertBRG2RGBA");
}

'File written in /content/src/cuda_launcher.h'

### MAIN EXPERIMENT 
Try all the previous code, with the following main. If you did not finish all the previous code, this file will show some execution errors.

The code is divided in two parts, one to define the parameters of the experiment and the other one is the main function with the experiment it self.

The experiment is the code that creates a BRG image in CPU, allocates GPU memory, copies the BRG image to GPU memory, and executes a GPU kernel to convert the BRG image into a RGBA image. The output of the kernel is another GPU pointer, so after the kernel execution, we have to copy back the results.

In [11]:
%%cuda --name experiment_settings.h
#pragma once
#define WIDTH 3840
#define HEIGHT 2160
#define EXPERIMENT_ITERATIONS 1

'File written in /content/src/experiment_settings.h'

In [12]:
%%cuda --name experiment.h
#include <cuda.h>
#include <chrono>
#include "/content/src/utils.h"
#include "/content/src/memory_functions.h"
#include "/content/src/check_results.h"
#include "/content/src/cuda_launcher.h"
#include "/content/src/experiment_settings.h"

void executeExperiment() {
  uchar3 *h_brg, *d_brg;
  uchar4 *h_rgba, *d_rgba;

  int bar_widht = HEIGHT/3;

  // Alloc and generate BRG bars.
  h_brg = (uchar3*)malloc(sizeof(uchar3)*WIDTH*HEIGHT);
  for (int i=0; i < WIDTH * HEIGHT; ++i) {
    if (i < bar_widht) {
      uchar3 temp = {255, 0, 0};
      h_brg[i] = temp; 
    } else if (i < bar_widht*2) {
      uchar3 temp = {0, 255, 0};
      h_brg[i] = temp;
    } else { 
      uchar3 temp = {0, 0, 255};
      h_brg[i] = temp;
    }
  }

  // Alloc RGBA pointers
  h_rgba = (uchar4*)malloc(sizeof(uchar4)*WIDTH*HEIGHT);

  // Alloc gpu pointers
  allocGPUData(WIDTH, HEIGHT, &d_brg, &d_rgba);
  
  // Prepare and copy data to GPU
  copyAndInitializeGPUData(WIDTH, HEIGHT, h_brg, d_brg, d_rgba);

  // Execute the GPU kernel
  executeKernelconvertBRG2RGBA(WIDTH, HEIGHT, d_brg, d_rgba, EXPERIMENT_ITERATIONS);

  // Copy data back from GPU to CPU, without streams
  CU_CHECK2(cudaMemcpy(h_rgba, d_rgba, sizeof(uchar4)*WIDTH*HEIGHT, cudaMemcpyDeviceToHost), "Cuda memcpy Device to Host: ");
    
  // Check results
  bool ok = checkResults(h_rgba, h_brg, WIDTH*HEIGHT);
  if (ok) {
      std::cout << "Executed!! Results OK." << std::endl;
  } else {
      std::cout << "Executed!! Results NOT OK." << std::endl;
  }

  // Free CPU pointers
  free(h_rgba);
  free(h_brg);

  // Free cuda pointers
  freeCUDAPointers(d_brg, d_rgba);
}

'File written in /content/src/experiment.h'

In [13]:
%%cu
#include "/content/src/experiment.h"
int main() {

  executeExperiment();

  return 0;
}

convertBRG2RGBA time for 1 iterations = 303us
First position x=0 y=0 z=255 w=255
Executed!! Results OK.



##Section 2:
Implement a version of the kernel and launcher that uses a one dimensional cuda GRID. That is, there is no more x and y, only x.

Modify the code below, click play, and then click play in the Main Experiment block, in Section 1.

Try different values of BLOCK_SIZE.

Check if there is any execution time improvement, compared to Section 1.


In [14]:
%%cuda --name cuda_launcher.h

#define BLOCK_SIZE 256

// UNIDIMENSIONAL KERNEL
__global__ void convertBRG2RGBA(uchar3 *brg, uchar4* rgba, int width, int height) {
  int x = threadIdx.x + (blockIdx.x * blockDim.x);//Use the thread id and block id's to compute x 
  
	// Protection to avoid segmentation fault
	if (x < width * height) {	
	    rgba[x].x = brg[x].y;
	    rgba[x].y = brg[x].z;
	    rgba[x].z = brg[x].x;
	    rgba[x].w = 255;
	}
}

void executeKernelconvertBRG2RGBA(int width, int height, uchar3* d_brg, uchar4* d_rgba, int numIters, cudaStream_t stream=0) {
  // Execute the GPU kernel
  dim3 block(BLOCK_SIZE, 1, 1);
  dim3 grid(ceil(width*height/(float)block.x), 1, 1);

  // A trick to avoid some undesired optimizations, that will not happen in real applications
  uchar3* d_dataVector[2];
  uchar3* d_secondData;
  CU_CHECK2(cudaMalloc(&d_secondData, width * height * sizeof(uchar3)), "cudaMalloc in executeKernelconvertBRG2RGBA");

  d_dataVector[0] = d_brg;
  d_dataVector[1] = d_secondData;

  CU_CHECK2(cudaMemcpyAsync(d_secondData, d_brg, width * height * sizeof(uchar3), cudaMemcpyDeviceToDevice, stream), "cudaMemcpyAsync in executeKernelconvertBRG2RGBA");

  auto t1 = std::chrono::high_resolution_clock::now();
  for (int i=0; i<numIters; ++i) {
    convertBRG2RGBA<<<grid, block, 0, stream>>>(d_dataVector[i%2], d_rgba, width, height);
  }
  CU_CHECK2(cudaDeviceSynchronize(), "cudaDeviceSynchronize:");
  auto t2 = std::chrono::high_resolution_clock::now();
  auto duration = std::chrono::duration_cast<std::chrono::microseconds>( t2 - t1 ).count();
  std::cout << "convertBRG2RGBA time for " << numIters << " iterations = "<< duration << "us" << std::endl;
}

'File written in /content/src/cuda_launcher.h'

In [15]:
%%cu
#include "/content/src/experiment.h"
int main() {

  executeExperiment();

  return 0;
}

convertBRG2RGBA time for 1 iterations = 277us
First position x=0 y=0 z=255 w=255
Executed!! Results OK.



Change the experiment settings, by executing more iterations and compare the unidimensional kernel with the bidimensional kernel.

Comment the results in the report.

### Section 2 comments:


|             | ONE-DIMENSION | TWO-DIMENSION |
|-------------|---------------|---------------|
| 100_ITER    | 39869us       | 48689us       |
| 500_ITER    | 198417us      | 242573us      |
| 1000_ITER   | 369619us      | 409701us      |
| 5000_ITER   | 1810931us     | 1856956us     |
| 10000_ITER  | 3596504us     | 3699985us     |
| 100000_ITER | 35726420us    | 36432094us    |


-----------------------------------------------

-Resizing image

|             | ONE-DIMENSION | TWO-DIMENSION |
|-------------|---------------|---------------|
| ORIGINAL    | 39869us       | 48689us       |
| + 10%       | 48167us       | 56904us       |
| + 20%       | 57257us       | 69865us       |
| - 10%       | 32363us       | 39336us       |
| - 20%       | 25676us       | 31188us       |


In [16]:
%%cuda --name experiment_settings.h
#pragma once
#define WIDTH 3840
#define HEIGHT 2160
#define EXPERIMENT_ITERATIONS 100 //try different values

'File written in /content/src/experiment_settings.h'

## Section 3:

Starting from Section 2, (use the best BLOCK_SIZE you found) try to optimize the memory accesses in some way, without using shared memory.

Comment in the report which memory access problems you observe. Are the memory accesses aligned, and therfore coalesced?

Remember that opposite to what the CPU compilers do, the nvcc compiler does not optimize the memory accesses in structs

Remember also that GPU memory is organized in blocks of 4 bytes, and any array based on data elements that are not multiple of 2, will not be alligned. To be coalesced (specially in old architectures), it also has to be multiple of 4.

In [17]:
%%cuda --name cuda_launcher.h

#define BLOCK_SIZE 256

// UNIDIMENSIONAL KERNEL BETTER MEMORY ACCESS
__global__ void convertBRG2RGBA(uchar3 *brg, uchar4* rgba, int width, int height) {
  int x = threadIdx.x + (blockIdx.x * blockDim.x);//Use the same code as in section 2 in this line
  
	// Protection to avoid segmentation fault
	if (x < width * height) {	
      uchar3 tmp_3 = brg[x];

      uchar4 tmp_4;
      
      tmp_4.x = tmp_3.y;
      tmp_4.y = tmp_3.z;
      tmp_4.z = tmp_3.x;
      tmp_4.w = 255;
      
      rgba[x] = tmp_4;
	}
}

void executeKernelconvertBRG2RGBA(int width, int height, uchar3* d_brg, uchar4* d_rgba, int numIters, cudaStream_t stream=0) {
  // Execute the GPU kernel
  dim3 block(BLOCK_SIZE, 1, 1);
  dim3 grid(ceil(width*height/(float)block.x), 1, 1);

  // A trick to avoid some undesired optimizations, that will not happen in real applications
  uchar3* d_dataVector[2];
  uchar3* d_secondData;
  CU_CHECK2(cudaMalloc(&d_secondData, width * height * sizeof(uchar3)), "cudaMalloc in executeKernelconvertBRG2RGBA");

  d_dataVector[0] = d_brg;
  d_dataVector[1] = d_secondData;

  CU_CHECK2(cudaMemcpyAsync(d_secondData, d_brg, width * height * sizeof(uchar3), cudaMemcpyDeviceToDevice, stream), "cudaMemcpyAsync in executeKernelconvertBRG2RGBA");

  auto t1 = std::chrono::high_resolution_clock::now();
  for (int i=0; i<numIters; ++i) {
    convertBRG2RGBA<<<grid, block, 0, stream>>>(d_dataVector[i%2], d_rgba, width, height);
  }
  CU_CHECK2(cudaDeviceSynchronize(), "cudaDeviceSynchronize:");
  auto t2 = std::chrono::high_resolution_clock::now();
  auto duration = std::chrono::duration_cast<std::chrono::microseconds>( t2 - t1 ).count();
  std::cout << "convertBRG2RGBA time for " << numIters << " iterations = "<< duration << "us" << std::endl;
}

'File written in /content/src/cuda_launcher.h'

In [19]:
%%cu
#include "/content/src/experiment.h"
int main() {

  executeExperiment();

  return 0;
}

convertBRG2RGBA time for 100 iterations = 14364us
First position x=0 y=0 z=255 w=255
Executed!! Results OK.



##Section 4:
Now optimize the GPU memory accesses so that each thread always reads at least one element of 4 bytes. Use shared memory to that end.

Look at the PDF Lab2CUDA in the campus, an read the last two pages. There you have a graphical explanation of the kernel issues. For this section you only need to understand the first figure.

About shared memory: we will refresh some concepts.

Shared memory, is a kind of memory that is visible only by the cuda threads of a thread block. Cuda threads from different thread blocks can not see the shared memory of other threadblocks.

Shared memory is a limited resource. Depending on the GPU model, you may have from 32KB to 64KB of shared memory. Additionally, this memory is not used only by one threadblock. It is partitioned in as many independent blocks as thread blocks can execute in a single Streaming Multiprocessor (check the documentation if you don't know what a SM is). 

So when you are defining the amount of shared memory you want, you are defining the amount of memory, every thread block will have available.

If you reserve 64KB of shared memory, in a GPU that has this capacity, only one thread block will execute on each SM, which is super slow. Each SM can concurrently execute from 8 to 32 thread blocks. For the best performance, you usually want the greatest amount of thread blocks active on each SM.

Therefore, you what to use the least shared memory possible, and only use it when it has clear benefits.


In [20]:
%%cuda --name cuda_launcher.h
#include "/content/src/experiment_settings.h"

// Try different vaues of BLOCK_SIZE
#define BLOCK_SIZE 256

// Number of 4 byte elements that we can make out of BLOCK_SIZE elements of 3 bytes
#define N_ELEMS_3_4_TBLOCK (BLOCK_SIZE * 3)/4
#define N_ELEMS_3_4_IMAGE (WIDTH*HEIGHT * 3)/4

// UNIDIMENSIONAL KERNEL SHARED MEMORY
__global__ void convertBRG2RGBA(uchar3 *brg, uchar4* rgba, int width, int height) {
  int position = threadIdx.x + (blockIdx.x * N_ELEMS_3_4_TBLOCK); // use N_ELEMS_3_4_TBLOCK to compute the position of each thread when we read brg as if it had elements of 4 bytes
  __shared__ uchar4 bgrShared[N_ELEMS_3_4_TBLOCK];
  
  if(threadIdx.x < N_ELEMS_3_4_TBLOCK && position < N_ELEMS_3_4_IMAGE) {
      uchar4* temp = reinterpret_cast<uchar4*>(brg);
      bgrShared[threadIdx.x] = temp[position];
  }

  __syncthreads();
  
  position = threadIdx.x + (blockIdx.x * blockDim.x);// recompute position without N_ELEMS_3_4_TBLOCK to write the results
	// Protection to avoid segmentation fault
	if (position < width*height) {	
        uchar3 local = reinterpret_cast<uchar3*>(bgrShared)[threadIdx.x];
        rgba[position] = make_uchar4(local.y,local.z,local.x,255);
	}
}

void executeKernelconvertBRG2RGBA(int width, int height, uchar3* d_brg, uchar4* d_rgba, int numIters, cudaStream_t stream=0) {
  // Execute the GPU kernel
  dim3 block(BLOCK_SIZE, 1, 1);
  dim3 grid(ceil(width*height/(float)block.x), 1, 1);

  // A trick to avoid some undesired optimizations, that will not happen in real applications
  uchar3* d_dataVector[2];
  uchar3* d_secondData;
  CU_CHECK2(cudaMalloc(&d_secondData, width * height * sizeof(uchar3)), "cudaMalloc in executeKernelconvertBRG2RGBA");

  d_dataVector[0] = d_brg;
  d_dataVector[1] = d_secondData;

  CU_CHECK2(cudaMemcpyAsync(d_secondData, d_brg, width * height * sizeof(uchar3), cudaMemcpyDeviceToDevice, stream), "cudaMemcpyAsync in executeKernelconvertBRG2RGBA");

  auto t1 = std::chrono::high_resolution_clock::now();
  for (int i=0; i<numIters; ++i) {
    convertBRG2RGBA<<<grid, block, 0, stream>>>(d_dataVector[i%2], d_rgba, width, height);
  }
  CU_CHECK2(cudaDeviceSynchronize(), "cudaDeviceSynchronize:");
  auto t2 = std::chrono::high_resolution_clock::now();
  auto duration = std::chrono::duration_cast<std::chrono::microseconds>( t2 - t1 ).count();
  std::cout << "convertBRG2RGBA time for " << numIters << " iterations = "<< duration << "us" << std::endl;
}

'File written in /content/src/cuda_launcher.h'

In [21]:
%%cu
#include "/content/src/experiment.h"
int main() {

  executeExperiment();

  return 0;
}

convertBRG2RGBA time for 100 iterations = 13297us
First position x=0 y=0 z=255 w=255
Executed!! Results OK.



##Section 5:

Now, following the explanation in the pdf document provided along with this colab, try to implement the described algorithm. Take into account that the piece of code that reads from temp variables and writes in pix_write, requires some changes.

In [22]:
%%cuda --name cuda_launcher.h
#include "/content/src/experiment_settings.h"

// Try different vaues of BLOCK_SIZE
#define BLOCK_SIZE 256

// Number of 4 byte elements that we can make out of BLOCK_SIZE elements of 3 bytes
#define N_ELEMS_3_4_TBLOCK (BLOCK_SIZE * 3)/4
#define N_ELEMS_3_4_IMAGE (WIDTH*HEIGHT * 3)/4
#define N_ELEMS_3_4_TBLOCK_3 N_ELEMS_3_4_TBLOCK/3

// UNIDIMENSIONAL KERNEL SHARED MEMORY
__global__ void convertBRG2RGBA(uchar3 *brg, uchar4* rgba, int width, int height) {
  int position = threadIdx.x + (blockIdx.x * N_ELEMS_3_4_TBLOCK); // use N_ELEMS_3_4_TBLOCK to compute the position of each thread when we read brg as if it had elements of 4 bytes
  __shared__ uchar4 bgrShared[N_ELEMS_3_4_TBLOCK];
  
  if(threadIdx.x < N_ELEMS_3_4_TBLOCK && position < N_ELEMS_3_4_IMAGE) {
      uchar4* temp = reinterpret_cast<uchar4*>(brg);
      bgrShared[threadIdx.x] = temp[position];
  }

  __syncthreads();
  
  // Vector de memoria compartida
  __shared__ uchar4 pix_write[1024];

	// Cada 3 threads escribim els bytes a memoria compartida
	if (threadIdx.x < N_ELEMS_3_4_TBLOCK_3) {
      
      // Guardem els bloc en registres temporals
      uchar4 tmp1 = bgrShared[3 * threadIdx.x];
      uchar4 tmp2 = bgrShared[3 * threadIdx.x + 1];
      uchar4 tmp3 = bgrShared[3 * threadIdx.x + 2];

      int pos = threadIdx.x + (blockIdx.x * N_ELEMS_3_4_TBLOCK_3);

      // Assignem les modificaions al vector de memoria compartida
      pix_write[threadIdx.x * 4]     = make_uchar4(tmp1.y, tmp1.z, tmp1.x, 255);
      pix_write[threadIdx.x * 4 + 1] = make_uchar4(tmp2.x, tmp2.y, tmp1.w, 255);
      pix_write[threadIdx.x * 4 + 2] = make_uchar4(tmp2.w, tmp3.x, tmp2.z, 255);
      pix_write[threadIdx.x * 4 + 3] = make_uchar4(tmp3.z, tmp3.w, tmp3.y, 255);

      // Apliquem la modificacio
      ((uint4*)rgba)[pos] = ((uint4*)pix_write)[threadIdx.x];
	}
}

void executeKernelconvertBRG2RGBA(int width, int height, uchar3* d_brg, uchar4* d_rgba, int numIters, cudaStream_t stream=0) {
  // Execute the GPU kernel
  dim3 block(BLOCK_SIZE, 1, 1);
  dim3 grid(ceil(width*height/(float)block.x), 1, 1);

  // A trick to avoid some undesired optimizations, that will not happen in real applications
  uchar3* d_dataVector[2];
  uchar3* d_secondData;
  CU_CHECK2(cudaMalloc(&d_secondData, width * height * sizeof(uchar3)), "cudaMalloc in executeKernelconvertBRG2RGBA");

  d_dataVector[0] = d_brg;
  d_dataVector[1] = d_secondData;

  CU_CHECK2(cudaMemcpyAsync(d_secondData, d_brg, width * height * sizeof(uchar3), cudaMemcpyDeviceToDevice, stream), "cudaMemcpyAsync in executeKernelconvertBRG2RGBA");

  auto t1 = std::chrono::high_resolution_clock::now();
  for (int i=0; i<numIters; ++i) {
    convertBRG2RGBA<<<grid, block, 0, stream>>>(d_dataVector[i%2], d_rgba, width, height);
  }
  CU_CHECK2(cudaDeviceSynchronize(), "cudaDeviceSynchronize:");
  auto t2 = std::chrono::high_resolution_clock::now();
  auto duration = std::chrono::duration_cast<std::chrono::microseconds>( t2 - t1 ).count();
  std::cout << "convertBRG2RGBA time for " << numIters << " iterations = "<< duration << "us" << std::endl;
}

'File written in /content/src/cuda_launcher.h'

In [23]:
%%cu
#include "/content/src/experiment.h"
int main() {

  executeExperiment();

  return 0;
}

convertBRG2RGBA time for 100 iterations = 13343us
First position x=0 y=0 z=255 w=255
Executed!! Results OK.



##Section 6:

Change all the host code necessary, to use cuda streams. Here you have an example.

In [24]:
%%cu
#include <iostream>

__global__ void square(int* d_input, int* d_output) {
    int x = threadIdx.x + blockIdx.x * blockDim.x;

    int val = d_input[x];
    // We exploit the temporal locality of the value stored in d_output[x]
    d_output[x] = val*val;
}

static const size_t dataSize = sizeof(int)*1024;

int main() {
    
    int *h_input, *h_output;
    h_input = (int*)malloc(dataSize);
    h_output = (int*)malloc(dataSize);

    for (int i=0; i<1024; ++i) h_input[i]=i;

    int *d_input, *d_output;
    cudaMalloc(&d_input, dataSize);
    cudaMalloc(&d_output, dataSize);

    cudaStream_t stream;
    cudaStreamCreate(&stream);

    dim3 block(512);
    dim3 grid(2);

    // The CPU thread does not wait that any of the following actions finish
    // It only asks the GPU to do the copies and the kernel and continues
    cudaMemcpyAsync(d_input, h_input, dataSize, cudaMemcpyHostToDevice, stream);
    square<<<grid, block, 0, stream>>>(d_input, d_output);
    cudaMemcpyAsync(h_output, d_output, dataSize, cudaMemcpyDeviceToHost, stream);

    // Here, we wait for all the orders enqueued in stream, to finish.
    cudaStreamSynchronize(stream);

    bool correct = true;
    for (int i=0; i<1024; ++i) correct &= h_output[i] == i*i;

    std::cout << "Finished and results are " << (correct ? "correct." : "not correct.") << std::endl;

    cudaStreamDestroy(stream);
    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    free(h_output);

    return 0;
}

Finished and results are correct.



Modify this code, to use streams

In [25]:
%%cuda --name cuda_launcher.h
#include "/content/src/experiment_settings.h"

// Try different vaues of BLOCK_SIZE
#define BLOCK_SIZE 256

// Number of 4 byte elements that we can make out of BLOCK_SIZE elements of 3 bytes
#define N_ELEMS_3_4_TBLOCK (BLOCK_SIZE * 3)/4
#define N_ELEMS_3_4_IMAGE (WIDTH*HEIGHT * 3)/4

// UNIDIMENSIONAL KERNEL SHARED MEMORY
__global__ void convertBRG2RGBA(uchar3 *brg, uchar4* rgba, int width, int height) {
  int position = threadIdx.x + (blockIdx.x * N_ELEMS_3_4_TBLOCK);// use N_ELEMS_3_4_TBLOCK to compute the position of each thread when we read brg as if it had elements of 4 bytes
  __shared__ uchar4 bgrShared[N_ELEMS_3_4_TBLOCK];
  
  if(threadIdx.x < N_ELEMS_3_4_TBLOCK && position < N_ELEMS_3_4_IMAGE) {
      uchar4* temp = reinterpret_cast<uchar4*>(brg);
      bgrShared[threadIdx.x] = temp[position];
  }

  __syncthreads();
  
  position = threadIdx.x + (blockIdx.x * blockDim.x);// recompute position without N_ELEMS_3_4_TBLOCK to write the results
	// Protection to avoid segmentation fault
	if (position < width*height) {	
        uchar3 local = reinterpret_cast<uchar3*>(bgrShared)[threadIdx.x];
        rgba[position] = make_uchar4(local.y,local.z,local.x,255);
	}
}

void executeKernelconvertBRG2RGBA(int width, int height, uchar3* d_brg, uchar4* d_rgba, int numIters, cudaStream_t stream=0) {
  // Execute the GPU kernel
  dim3 block(BLOCK_SIZE, 1, 1);
  dim3 grid(ceil(width*height/(float)block.x), 1, 1);

  // A trick to avoid some undesired optimizations, that will not happen in real applications
  uchar3* d_dataVector[2];
  uchar3* d_secondData;
  CU_CHECK2(cudaMalloc(&d_secondData, width * height * sizeof(uchar3)), "cudaMalloc in executeKernelconvertBRG2RGBA");

  d_dataVector[0] = d_brg;
  d_dataVector[1] = d_secondData;

  CU_CHECK2(cudaMemcpyAsync(d_secondData, d_brg, width * height * sizeof(uchar3), cudaMemcpyDeviceToDevice, stream), "cudaMemcpyAsync in executeKernelconvertBRG2RGBA");

  auto t1 = std::chrono::high_resolution_clock::now();
  for (int i=0; i<numIters; ++i) {
    convertBRG2RGBA<<<grid, block, 0, stream>>>(d_dataVector[i%2], d_rgba, width, height);
  }
  CU_CHECK2(cudaDeviceSynchronize(), "cudaDeviceSynchronize:");
  auto t2 = std::chrono::high_resolution_clock::now();
  auto duration = std::chrono::duration_cast<std::chrono::microseconds>( t2 - t1 ).count();
  std::cout << "convertBRG2RGBA time for " << numIters << " iterations = "<< duration << "us" << std::endl;
}

'File written in /content/src/cuda_launcher.h'

In [26]:
%%cuda --name memory_functions.h
void allocGPUData(int width, int height, uchar3** d_brg, uchar4** d_rgba){
  // Alloc gpu pointers
  CU_CHECK2(cudaMalloc(d_brg, sizeof(uchar3)*width*height), "Alloc d_brg:");
  // Can you finish this one? Replace cudaSucces with the proper cuda API call
  CU_CHECK2(cudaMalloc(d_rgba, sizeof(uchar4)*width*height), "Alloc d_rgba:");
}
void copyAndInitializeGPUData(int width, int height, uchar3* h_brg, uchar3* d_brg, uchar4* d_rgba, cudaStream_t stream=0) {
  // Copy data to GPU
  CU_CHECK2(cudaMemcpyAsync(d_brg, h_brg, width*height*sizeof(uchar3), cudaMemcpyHostToDevice, stream), "Copy h_brg to d_brg:");
  // Init output buffer to 0
  CU_CHECK2(cudaMemsetAsync(d_rgba, 0, width*height*sizeof(uchar4), stream), "Memset d_rgba:");
}
void freeCUDAPointers(uchar3* d_brg, uchar4* d_rgba) {
  // Free cuda pointers. Replace the cudaErrorInvalidValue flag
  // with the proper cuda API call, to free the GPU pointers
  CU_CHECK2(cudaFree(d_brg), "Cuda free d_bgr:");
  CU_CHECK2(cudaFree(d_rgba), "Cuda free d_rgba:");
  // Clean GPU device
  CU_CHECK2(cudaDeviceReset(), "Cuda device reset:");
}

'File written in /content/src/memory_functions.h'

And also modify this code, so that mem copies from CPU to GPU and from GPU to CPU use an stream, and are not blocking.

Additionally, add a chrono between the first memcpy (included) and the cudaStreamSynchronize. This is the time you will have to compare.

Follow the indications in the code.

In [27]:
%%cuda --name experiment.h
#include <cuda.h>
#include <chrono>
#include "/content/src/utils.h"
#include "/content/src/memory_functions.h"
#include "/content/src/check_results.h"
#include "/content/src/cuda_launcher.h"
#include "/content/src/experiment_settings.h"

void executeExperiment() {
  uchar3 *h_brg, *d_brg;
  uchar4 *h_rgba, *d_rgba;

  int bar_widht = HEIGHT/3;

  // Alloc and generate BRG bars.
  h_brg = (uchar3*)malloc(sizeof(uchar3)*WIDTH*HEIGHT);
  for (int i=0; i < WIDTH * HEIGHT; ++i) {
    if (i < bar_widht) {
      uchar3 temp = {255, 0, 0};
      h_brg[i] = temp; 
    } else if (i < bar_widht*2) {
      uchar3 temp = {0, 255, 0};
      h_brg[i] = temp;
    } else { 
      uchar3 temp = {0, 0, 255};
      h_brg[i] = temp;
    }
  }

  cudaStream_t stream;
  cudaStreamCreate(&stream);

  // Alloc RGBA pointers
  h_rgba = (uchar4*)malloc(sizeof(uchar4)*WIDTH*HEIGHT);

  // Alloc gpu pointers
  allocGPUData(WIDTH, HEIGHT, &d_brg, &d_rgba);
  
  // Start measuring time here
  auto t1 = std::chrono::high_resolution_clock::now();
  copyAndInitializeGPUData(WIDTH, HEIGHT, h_brg, d_brg, d_rgba);

  // Execute the GPU kernel
  executeKernelconvertBRG2RGBA(WIDTH, HEIGHT, d_brg, d_rgba, EXPERIMENT_ITERATIONS, stream);

  // Copy data back from GPU to CPU
  CU_CHECK2(cudaMemcpyAsync(h_rgba, d_rgba, sizeof(uchar4)*WIDTH*HEIGHT, cudaMemcpyDeviceToHost, stream), "Cuda memcpy Device to Host: ");

  // Synchronize the stream here
  cudaStreamSynchronize(stream);

  // Stop measuring time here, and print it
  auto t2 = std::chrono::high_resolution_clock::now();
  auto duration = std::chrono::duration_cast<std::chrono::microseconds>( t2 - t1 ).count();
  std::cout << "Duration to compare = "<< duration << "us" << std::endl;


    
  // Check results
  bool ok = checkResults(h_rgba, h_brg, WIDTH*HEIGHT);
  if (ok) {
      std::cout << "Executed!! Results OK." << std::endl;
  } else {
      std::cout << "Executed!! Results NOT OK." << std::endl;
  }

  // Free CPU pointers
  free(h_rgba);
  free(h_brg);

  // Free cuda pointers
  freeCUDAPointers(d_brg, d_rgba);

  cudaStreamDestroy(stream);
}

'File written in /content/src/experiment.h'

Do the following:

1.   Use the fastest kernel version.
2.   Use number of iterations = 1.
3.   Compare the same kernel, with the original Host code, and this new Host code.
4.   To do so, you can use the code you do now, you only need to set stream=0 in order to simulate the original code.
5.   Execute it with the following code.
6.   Compare and try to explain the performance difference in the report.


In [28]:
%%cu
#include "/content/src/experiment.h"
int main() {

  executeExperiment();

  return 0;
}

convertBRG2RGBA time for 100 iterations = 13342us
Duration to compare = 28823us
First position x=0 y=0 z=255 w=255
Executed!! Results OK.



Section 6 comments:

|              | 1 ITERATION | 100 ITERATIONS|
|--------------|-------------|---------------|
|Original Host | 21468 us    | 54050 us      |
|Nwe Host      | 21148 us    |54138 us       |
